In [1]:
#coding=utf-8

import tensorflow as tf
import numpy as np
import time
import cifar10_input,cifar10
import functools
import os
from datetime import datetime
import math
data_dir = 'C:\cifar10\cifar10_data\cifar-10-batches-bin'  # data path

#batch_size
batch_size = 32

#function for initialization conv2d and pooling
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.05, shape = shape)
    return tf.Variable(initial)

conv2d_11 = functools.partial(tf.nn.conv2d,strides = [1,1,1,1],padding = 'SAME')
conv2d_22 = functools.partial(tf.nn.conv2d,strides = [1,2,2,1],padding = 'SAME')

max_pool22 = functools.partial(tf.nn.max_pool,ksize = [1,2,2,1],
                               strides= [1,2,2,1],padding = 'SAME')

#information for Tensorboard
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

#get data&label for train and test
with tf.device('/cpu:0'):
    image_t, label_t = cifar10_input.distorted_inputs(data_dir = data_dir, batch_size = batch_size)
    image_test, label_test = cifar10_input.inputs(eval_data = False,data_dir = data_dir, batch_size = batch_size)
    
#Input placeholder    
input_image = tf.placeholder(tf.float32, [batch_size, 24,24,3])
input_label = tf.placeholder(tf.int32,[batch_size])


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [15]:
#tf.app.flags.DEFINE_string('f', '', 'kernel')
#cifar10.maybe_download_and_extract()|
with tf.device('/cpu:0'):
    image_t, label_t = cifar10_input.distorted_inputs(data_dir = data_dir, batch_size = batch_size)
    image_test, label_test = cifar10_input.inputs(eval_data = False,data_dir = data_dir, batch_size = batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [16]:
input_image = tf.placeholder(tf.float32, [batch_size, 24,24,3])
input_label = tf.placeholder(tf.int32,[batch_size])

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.05, shape = shape)
    return tf.Variable(initial)

conv2d_11 = functools.partial(tf.nn.conv2d,strides = [1,1,1,1],padding = 'SAME')
conv2d_22 = functools.partial(tf.nn.conv2d,strides = [1,2,2,1],padding = 'SAME')

max_pool22 = functools.partial(tf.nn.max_pool,ksize = [1,2,2,1],
                               strides= [1,2,2,1],padding = 'SAME')

In [2]:
####SECTION ONE----------DEPTH64
##VGG16 layer1  In 24*24*3 Out 24*24*64   
W_1 = weight_variable([3,3,3,64])
b_1 = bias_variable([64])
conv1 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(input_image,W_1),b_1))

##VGG16 layer2 In 24*24*64 Out 24*24*64   
W_2 = weight_variable([3,3,64,64])
b_2 = bias_variable([64])
conv2 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv1,W_2),b_2))

##VGG16 layer3 In 24*24*64 Out 12*12*64   
pool3 = max_pool22(conv2)



####SECTION TWO----------DEPTH128
##VGG16 layer4 In 12*12*64 Out 12*12*128   
W_4 = weight_variable([3,3,64,128])
b_4 = bias_variable([128])
conv4 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(pool3,W_4),b_4))

##VGG16 layer5 In 12*12*128 Out 12*12*128 
W_5 = weight_variable([3,3,128,128])
b_5 = bias_variable([128])
conv5 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv4,W_5),b_5))

##VGG16 layer6 In 12*12*128 Out 6*6*128  
pool6 = max_pool22(conv5)



####SECTION THREE----------DEPTH256
##VGG16 layer7 In 6*6*128 Out 6*6*256 
W_7 = weight_variable([3,3,128,256])
b_7 = bias_variable([256])
conv7 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(pool6,W_7),b_7))

##VGG16 layer8 In 6*6*256 Out 6*6*256 
W_8 = weight_variable([3,3,256,256])
b_8 = bias_variable([256])
conv8 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv7,W_8),b_8))

##VGG16 layer9 In 6*6*256 Out 6*6*256 
W_9 = weight_variable([3,3,256,256])
b_9 = bias_variable([256])
conv9 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv8,W_9),b_9))

##VGG16 layer10 In 6*6*256 Out 3*3*256  
pool10 = max_pool22(conv9)



####SECTION FOUR----------DEPTH512
##VGG16 layer11 In 3*3*256 Out 3*3*512 
W_11 = weight_variable([3,3,256,512])
b_11 = bias_variable([512])
conv11 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(pool10,W_11),b_11))

##VGG16 layer8 In 3*3*512 Out 3*3*512 
W_12 = weight_variable([3,3,512,512])
b_12 = bias_variable([512])
conv12 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv11,W_12),b_12))

##VGG16 layer9 In 3*3*512 Out 3*3*512 
W_13 = weight_variable([3,3,512,512])
b_13 = bias_variable([512])
conv13 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv12,W_13),b_13))

##VGG16 layer10 In 3*3*512 Out 1*1*512  
pool14 = max_pool22(conv13)



####SECTION FIVE----------DEPTH512
##VGG16 layer15 In 1*1*256 Out 1*1*512 
W_15 = weight_variable([1,1,512,512])
b_15 = bias_variable([512])
conv15 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(pool14,W_15),b_15))

##VGG16 layer16 In 3*3*512 Out 3*3*512 
W_16 = weight_variable([1,1,512,512])
b_16 = bias_variable([512])
conv16 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv15,W_16),b_16))

##VGG16 layer17 In 3*3*512 Out 3*3*512 
W_17 = weight_variable([1,1,512,512])
b_17 = bias_variable([512])
conv17 = tf.nn.relu6(tf.nn.bias_add(conv2d_11(conv16,W_17),b_17))


####SECTION SIX----------FULLY CONNECT
##VGG16 Layer18 fully connected layer in512 our4096
F_18 = tf.reshape(conv17,[batch_size,-1])
length = F_18.get_shape()[1].value
W_18 = weight_variable([length, 4096])
b_18 = bias_variable([4096])
FULLY18 = tf.add(tf.matmul(F_18,W_18),b_18)

##VGG16 Layer18 fully connected layer in512 our4096
F_19 = tf.reshape(conv17,[batch_size,-1])
length = F_19.get_shape()[1].value
W_19 = weight_variable([length, 4096])
b_19 = bias_variable([4096])
FULLY19 = tf.add(tf.matmul(F_19,W_19),b_19)

##VGG16 Layer18 fully connected layer in512 our4096
F_20 = tf.reshape(conv17,[batch_size,-1])
length = F_18.get_shape()[1].value
W_20 = weight_variable([length, 1000])
b_20 = bias_variable([1000])
prediction = tf.add(tf.matmul(F_20,W_20),b_20)


In [3]:
##Loss
label = tf.cast(input_label,tf.int64)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label,
                                                              logits=prediction,
                                                              name = 'cross_entropy')
cross_entropy_mean = tf.reduce_mean(cross_entropy)

#Study Step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_mean) 

#Precision
top_k_op = tf.nn.in_top_k(prediction, input_label, 1)
true_sample = 0
step = 0

#Tensorboard
loss_train_summary = tf.summary.scalar('loss',cross_entropy_mean)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

#Initiazliaztion
init = tf.global_variables_initializer()

In [4]:
with tf.Session() as sess:
    sess.run(init)
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(sess=sess, coord=coord)
    #tf.train.start_queue_runners()
    for i in range(80000): 
        image_batch, label_batch = sess.run([image_t, label_t]) # 获取训练数据    
        _,loss = sess.run([train_step,cross_entropy_mean], feed_dict={input_image: image_batch, input_label: label_batch})  
        pred = sess.run([top_k_op],feed_dict={input_image: image_batch, input_label: label_batch})
        
        true_sample += np.sum(pred)
        step += batch_size
        if(i%100 ==0):
            print('loss is', loss)
            print('accuracy is',float(true_sample/step))
            true_sample = 0
            step = 0
            
            #Tensorboard
            summary_str_train = loss_train_summary.eval(feed_dict = {input_image: image_batch, input_label: label_batch})
            file_writer.add_summary(summary_str_train,i)
            
            
    #test
    num_examples = 10000
    num_iter = int(math.ceil(num_examples / batch_size))
    true_test_sample = 0
    total_sample_count = num_iter * batch_size
    step = 0
    while step < num_iter:
        image_batch, label_batch = sess.run([image_test, label_test])
        pred = sess.run([top_k_op],feed_dict={input_image: image_batch, input_label: label_batch})
        true_test_sample += np.sum(pred)
        step += 1
        if (step%50 == 0):
            print('precision at this stage:',true_test_sample/ (step* batch_size))

    precision = true_test_sample / total_sample_count
    print('Final Precision =%.3f' % precision)
    
    coord.request_stop() 
    coord.join() 
    
file_writer.close()

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
loss is 41.574577
accuracy is 0.0
loss is 2.9662688
accuracy is 0.311875
loss is 2.5002902
accuracy is 0.2871875
loss is 2.5005345
accuracy is 0.255
loss is 2.3655486
accuracy is 0.243125
loss is 2.252229
accuracy is 0.2465625
loss is 2.2201076
accuracy is 0.2375
loss is 2.1433284
accuracy is 0.2490625
loss is 2.3659105
accuracy is 0.2315625
loss is 2.0936131
accuracy is 0.236875
loss is 2.0843885
accuracy is 0.2459375
loss is 2.1443434
accuracy is 0.25625
loss is 2.2686539
accuracy is 0.2478125
loss is 1.9766241
accuracy is 0.2709375
loss is 2.4086733
accuracy is 0.26375
loss is 2.2938857
accuracy is 0.2559375
loss is 2.441967
accuracy is 0.2665625
loss is 2.0357409
accuracy is 0.2821875
loss is 2.0992117
accuracy is 0.27375
loss is 2.4148898
accuracy is 0.25375
loss is 1.9718112
accuracy is 0.255
loss is 1.9254024
accuracy is 0.2640625
loss is 2.37979
accuracy is 0.25375
loss is 2.09369
accuracy is 0.2

loss is 1.6691903
accuracy is 0.429375
loss is 1.6398926
accuracy is 0.4009375
loss is 1.6421964
accuracy is 0.4325
loss is 1.6778007
accuracy is 0.424375
loss is 1.4919839
accuracy is 0.4559375
loss is 1.3474989
accuracy is 0.4490625
loss is 1.4894423
accuracy is 0.4590625
loss is 1.4920411
accuracy is 0.4353125
loss is 1.4414792
accuracy is 0.4328125
loss is 1.7815781
accuracy is 0.42875
loss is 1.6281464
accuracy is 0.429375
loss is 1.843272
accuracy is 0.450625
loss is 1.5418175
accuracy is 0.43625
loss is 1.4615694
accuracy is 0.438125
loss is 1.5463257
accuracy is 0.4284375
loss is 1.8784249
accuracy is 0.436875
loss is 1.520381
accuracy is 0.445
loss is 1.2933562
accuracy is 0.453125
loss is 1.3879375
accuracy is 0.48125
loss is 1.3855265
accuracy is 0.4490625
loss is 1.7881273
accuracy is 0.4553125
loss is 1.6452248
accuracy is 0.450625
loss is 1.3277105
accuracy is 0.4746875
loss is 1.4470062
accuracy is 0.4753125
loss is 1.3041563
accuracy is 0.4621875
loss is 1.475183
accura

loss is 1.0964423
accuracy is 0.606875
loss is 1.0923578
accuracy is 0.6240625
loss is 1.2068207
accuracy is 0.628125
loss is 1.3265873
accuracy is 0.6134375
loss is 0.8982372
accuracy is 0.63625
loss is 1.4179255
accuracy is 0.609375
loss is 1.288367
accuracy is 0.6203125
loss is 1.4401108
accuracy is 0.6003125
loss is 1.155559
accuracy is 0.61625
loss is 1.2545731
accuracy is 0.62875
loss is 1.0512891
accuracy is 0.6225
loss is 1.2125893
accuracy is 0.639375
loss is 1.0326049
accuracy is 0.63875
loss is 1.5179288
accuracy is 0.6140625
loss is 1.2125307
accuracy is 0.615
loss is 1.292141
accuracy is 0.6359375
loss is 0.86483634
accuracy is 0.6115625
loss is 0.859846
accuracy is 0.6140625
loss is 1.044065
accuracy is 0.63
loss is 1.2480596
accuracy is 0.6215625
loss is 1.3695486
accuracy is 0.6228125
loss is 1.1293696
accuracy is 0.6259375
loss is 1.2302749
accuracy is 0.6303125
loss is 1.5048759
accuracy is 0.61875
loss is 1.2425629
accuracy is 0.6315625
loss is 0.9667389
accuracy is 

loss is 1.2913132
accuracy is 0.6915625
loss is 0.868064
accuracy is 0.6909375
loss is 1.1974901
accuracy is 0.6846875
loss is 0.84315443
accuracy is 0.6925
loss is 0.89736843
accuracy is 0.71
loss is 1.0786314
accuracy is 0.6996875
loss is 0.8731327
accuracy is 0.6871875
loss is 0.8047875
accuracy is 0.7053125
loss is 0.92643535
accuracy is 0.70625
loss is 0.8873229
accuracy is 0.7190625
loss is 0.8619637
accuracy is 0.695625
loss is 1.0567343
accuracy is 0.6825
loss is 1.1976559
accuracy is 0.704375
loss is 1.0963112
accuracy is 0.703125
loss is 0.6528725
accuracy is 0.7096875
loss is 0.76524526
accuracy is 0.7178125
loss is 1.2412988
accuracy is 0.69625
loss is 1.1259645
accuracy is 0.6909375
loss is 0.9954483
accuracy is 0.7040625
loss is 0.8944653
accuracy is 0.69
loss is 0.9890578
accuracy is 0.6953125
loss is 1.0040574
accuracy is 0.6884375
loss is 1.0839043
accuracy is 0.7071875
loss is 0.99091935
accuracy is 0.7146875
loss is 1.1052545
accuracy is 0.691875
loss is 0.92055905
a

In [ ]:
# test

#prepartion 
coord = tf.train.Coordinator()
tf.train.start_queue_runners(sess=sess, coord=coord)
#test
num_examples = 10000
num_iter = int(math.ceil(num_examples / batch_size))
true_test_sample = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([image_test, label_test])
    pred = sess.run([top_k_op],feed_dict={input_image: image_batch, input_label: label_batch})
    true_test_sample += np.sum(pred)
    step += 1
    if (step%50 == 0):
        print('precision at this stage:',true_test_sample/ (step* batch_size))

precision = true_test_sample / total_sample_count
print('Final Precision @ 1 =%.3f' % precision)
#end
coord.request_stop() # 请求线程结束
coord.join() # 等待线程结束

In [ ]:
##Loss
label = tf.cast(input_label,tf.int64)
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label,
                                                              logits=prediction,
                                                              name = 'cross_entropy')
cross_entropy_mean = tf.reduce_mean(cross_entropy)


##Study Step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy_mean) 

##Accuracy
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(input_label,1)) 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
accuracy_train = tf.reduce_mean(tf.cast(correct_prediction, "float"))


init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(50000):              
        batch_x, batch_y = mnist.train.next_batch(16)           
        sess.run(train_step, feed_dict={Input: batch_x, Label: batch_y})   
        if(i%100==0): 
            ##Output
            print ("test_accuracy:",sess.run(accuracy, feed_dict={input_image: mnist.test.images, input_label: mnist.test.labels}))
            print ("train_accuracy:",sess.run(accuracy, feed_dict={input_image: mnist.train.images, input_label: mnist.train.labels}))

file_writer.close()